# Process Json file to find labels for each image

In [1]:
#Open JSON file and store it in a DataFrame
import pandas as pd

filePath='F:/Data_BachelorHES/3.Video_CNC/8.FullyProcessedOCR/ocr_results_1.json'

with open(filePath, 'r') as f:
    #data = json.load(f)
    DfJson=pd.read_json(f)
    DfJson['Text_Box_(x,y,w,h)']=[DfJson['Detected_Text'][i]['Text_Box_(x,y,w,h)'] for i in range(len(DfJson))]
    DfJson['Text']=[DfJson['Detected_Text'][i]['Text'] for i in range(len(DfJson))]
    DfJson['Confidence']=[DfJson['Detected_Text'][i]['Confidence'] for i in range(len(DfJson))]
    DfJson['White_Ratio']=[DfJson['Detected_Text'][i]['White_Ratio'] for i in range(len(DfJson))]
    DfJson['FilePath']=[DfJson['Detected_Text'][i]['FilePath'] for i in range(len(DfJson))]

del f

   

In [2]:
from os import listdir

#Analyze the number of folders in the kaggle directory to ensure every frame has been processed
mypath='F:/Data_BachelorHES/3.Video_CNC/8.FullyProcessedOCR/kaggle_Images/'
myNames= listdir(mypath)
processedDirNames = [int(f) for f in myNames  if f.isdigit()]
processedDirNames.sort()
print("Number of folders resulting from kaggle notebook:",len(processedDirNames))
del myNames, mypath, processedDirNames

Number of folders resulting from kaggle notebook: 55117


In [3]:
#redefine the filepath from the json made in kaggle to the filepath on local machine
def update_file_paths(file_list, new_root_path):
    new_files=[]
    if len(file_list)==0:
        return new_files
    
    for files in file_list:
        myfile=files.split('/')[-2:]
        new_file_path=new_root_path + myfile[0] +'/' + myfile[1]
        new_files.append(new_file_path)
    return new_files

newRootPath='F:/Data_BachelorHES/3.Video_CNC/8.FullyProcessedOCR/kaggle_Images/'
DfJson['FilePath']=DfJson['FilePath'].apply(lambda file_list: update_file_paths(file_list, newRootPath))
del newRootPath

In [4]:
#Get only the relevant data
rawOCRData=[]
for _, row in DfJson.iterrows():
    row_data={'Frame_Number' : row['Frame_Number']}
    for i in range(len(row['Text'])):
        row_data[f'Text_{i}']=row['Text'][i]
    rawOCRData.append(row_data)
DfRawData=pd.DataFrame(rawOCRData)

del i, row, row_data 

In [5]:
#Check if a frame has not been processed
DfRawData.sort_values(by='Frame_Number', inplace=True)

prevRow=DfRawData.iloc[0]
nbDiff=0
for idx, row in DfRawData.iterrows():
    if idx  == 0 :
        continue
    diff=row['Frame_Number']-prevRow['Frame_Number']
    if diff != 5:
        nbDiff+=1
        print(f"Frame {row['Frame_Number']} has a difference of {diff} with previous frame {prevRow['Frame_Number']}")
    prevRow=row
print(f"Number of unprocessed frames : {nbDiff}")

del diff, idx, prevRow, row, nbDiff

Number of unprocessed frames : 0


In [6]:

text0NaData=[DfRawData.loc[l, :] for l in DfRawData.index if pd.isna(DfRawData.loc[l, 'Text_0'])]
print('Number of frames without a reading : ', len(text0NaData))
text3NotNaData=[DfRawData.loc[l, :] for l in DfRawData.index if pd.notna(DfRawData.loc[l, 'Text_3'])]
text2NotNaData=[DfRawData.loc[l, :] for l in DfRawData.index if pd.notna(DfRawData.loc[l, 'Text_2']) and pd.isna(
    DfRawData.loc[l, 'Text_3'])]
text1NotNaData=[DfRawData.loc[l, :] for l in DfRawData.index if pd.notna(DfRawData.loc[l, 'Text_1']) and pd.isna(
    DfRawData.loc[l, 'Text_2'])]
text0NotNaData=[DfRawData.loc[l, :] for l in DfRawData.index if pd.notna(DfRawData.loc[l, 'Text_0']) and pd.isna(
    DfRawData.loc[l, 'Text_1'])]

print("Frames with 1 readings :            ", len(text0NotNaData))
print("Frames with 2 readings :            ", len(text1NotNaData))
print("Frames with 3 readings :              ", len(text2NotNaData))
print("Frames with 4 readings :               ", len(text3NotNaData))

print("Total number of Frames :            ", len(text3NotNaData)+len(text2NotNaData)+len(text1NotNaData)+len(text0NotNaData)+len(text0NaData))

del text0NaData, text0NotNaData, text1NotNaData, text2NotNaData, text3NotNaData


Number of frames without a reading :  1825
Frames with 1 readings :             20905
Frames with 2 readings :             31939
Frames with 3 readings :               407
Frames with 4 readings :                41
Total number of Frames :             55117


In [7]:
#Out of data extract two lists : 
#1. A list of frames with their corresponding texts with missing texts filled with the previous frame's text
#2. A list of frames without a text reading

def create_frame_list(data):
    
    my_frames_texts=[]
    prev_frame=data[0]
    my_missing_frame_texts=[]
    j=0
    for i, frame in enumerate(data):
        len_txt=len(frame)
        if len_txt==1:
                j+=1
                print(f'No text detected for frame : {frame["Frame_Number"]}, copy of previous frame : {prev_frame["Frame_Number"]}, j={j}')
                my_missing_frame_texts.append(frame)
                my_frames_texts.append([frame["Frame_Number"],*my_frames_texts[i-1][1:]])
        elif len_txt==2:
                my_frames_texts.append([frame["Frame_Number"], frame["Text_0"]])
        elif len_txt==3:
                my_frames_texts.append([frame["Frame_Number"], frame["Text_0"], frame["Text_1"]])
        elif len_txt==4:
                my_frames_texts.append([frame["Frame_Number"], frame["Text_0"], frame["Text_1"], frame["Text_2"]])
        elif len_txt==5:
                my_frames_texts.append([frame["Frame_Number"], frame["Text_0"], frame["Text_1"], frame["Text_2"], frame["Text_3"]])
        else:
            print('Error in the number of texts detected')
        prev_frame=frame
    return my_frames_texts, my_missing_frame_texts
myFrameTexts, myMissingFrames=create_frame_list(rawOCRData)

No text detected for frame : 10645, copy of previous frame : 10640, j=1
No text detected for frame : 10665, copy of previous frame : 10660, j=2
No text detected for frame : 10675, copy of previous frame : 10670, j=3
No text detected for frame : 10680, copy of previous frame : 10675, j=4
No text detected for frame : 10685, copy of previous frame : 10680, j=5
No text detected for frame : 10735, copy of previous frame : 10730, j=6
No text detected for frame : 10750, copy of previous frame : 10745, j=7
No text detected for frame : 10760, copy of previous frame : 10755, j=8
No text detected for frame : 10790, copy of previous frame : 10785, j=9
No text detected for frame : 10800, copy of previous frame : 10795, j=10
No text detected for frame : 10805, copy of previous frame : 10800, j=11
No text detected for frame : 10815, copy of previous frame : 10810, j=12
No text detected for frame : 11120, copy of previous frame : 11115, j=13
No text detected for frame : 11125, copy of previous frame :

In [8]:
#Find contiguous missing frames in order to determine what is the longest time without text 
contigMissingFrames = {}
uniqueMissingFrames = []
prevF = myMissingFrames[0]
i = 0
for f in myMissingFrames:
    if f == prevF:
        continue
    if f['Frame_Number'] - prevF['Frame_Number'] == 5:
        if f'Group_{i}' not in contigMissingFrames or prevF['Frame_Number'] not in contigMissingFrames[f'Group_{i}']:
            i += 1
            contigMissingFrames[f'Group_{i}'] = []
            contigMissingFrames[f'Group_{i}'].append(prevF['Frame_Number'])
        contigMissingFrames[f'Group_{i}'].append(f['Frame_Number'])
    prevF = f
for i, f in enumerate(myMissingFrames):
    if i == 0 or i == (len(myMissingFrames) - 1):
        uniqueMissingFrames.append(f['Frame_Number'])
        continue
    if f['Frame_Number'] - myMissingFrames[i - 1]['Frame_Number'] > 5 and myMissingFrames[i + 1]['Frame_Number'] - f[
        'Frame_Number'] > 5:
        uniqueMissingFrames.append(f['Frame_Number'])
del f, i, prevF


In [10]:
#Find the longest time without text
maxLen=0
occur=0
avgLen=0
myLongestGroup=''
for key in contigMissingFrames:
    avgLen+=len(contigMissingFrames[key])
    if len(contigMissingFrames[key])>maxLen:
        maxLen=len(contigMissingFrames[key])
        myLongestGroup=key
        occur=1
    elif len(contigMissingFrames[key])==maxLen:
        occur+=1
totLen=avgLen
avgLen=avgLen/len(contigMissingFrames)        
        
print(' Max time without a text reading is :{} contiguous frames\n That is : {} seconds \n For the frames : {} \n It happens {} times\n Average number of missing frames is : {}\n Total number of groups : {}\n Total contiguous frames without a text reading : {} \n Total unique frames without a text reading :{}'.format(maxLen, maxLen*5/30, contigMissingFrames[myLongestGroup], occur, avgLen, len(contigMissingFrames), totLen, len(uniqueMissingFrames)))

del key, maxLen, myLongestGroup, occur, avgLen, totLen

 Max time without a text reading is :29 contiguous frames
 That is : 4.833333333333333 seconds 
 For the frames : [132995, 133000, 133005, 133010, 133015, 133020, 133025, 133030, 133035, 133040, 133045, 133050, 133055, 133060, 133065, 133070, 133075, 133080, 133085, 133090, 133095, 133100, 133105, 133110, 133115, 133120, 133125, 133130, 133135] 
 It happens 1 times
 Average number of missing frames is : 4.39068100358423
 Total number of groups : 279
 Total contiguous frames without a text reading : 1225 
 Total unique frames without a text reading :600


In [11]:
del uniqueMissingFrames, myMissingFrames, contigMissingFrames

In [28]:
DfFrames=pd.DataFrame(myFrameTexts, columns=['Frame_nb', 'OCR_1', 'OCR_2', 'OCR_3', 'OCR_4'])

## Import CNC Program lines for string matching

In [32]:
#Get the lines from the program
filepath="F:/Data_BachelorHES/3.Video_CNC/9.ProgramLines/programmeCNC_Trimmed.txt"
with open(filepath, "r", encoding="utf-8") as f:
    cncLines = f.readlines()

cncLines=[line.strip() for line in cncLines]
del f, filepath   

In [33]:
def recognize_line_number(previous_num, cnc_line):
    try:
        my_num=int(cnc_line.split(' ')[0].split('N')[-1])
    except ValueError:
        my_num= previous_num + 5
    return my_num
    

progLines=[]

myPrevNum=int(cncLines[0].split(' ')[0].split('N')[-1])
myOp=(3335,"(PRE-PERCAGE D4.2)")

for i, line in enumerate(cncLines):
    myNum = recognize_line_number(myPrevNum, line)

    if myNum%10==0:
        progLines.append((myNum, line, myOp[0], myOp[1]))
        myPrevNum=myNum
    else:
        myOp=(myNum, line)
        myPrevNum=myNum-5
    

del i, line, myNum, myPrevNum, myOp, cncLines

In [34]:
myTools=["T07 M06 (MECHE MD D4.2)"]
for i, line in enumerate(progLines):
    secondStr=line[1].split(' ')[1:]
    ' '.join(i for i in secondStr)
    if 'T' in secondStr[0] and len(secondStr)>1:
        print(' '.join(i for i in secondStr))
        myTools.append(' '.join(i for i in secondStr))
    
del i, line, secondStr

T02 M06 (VHM 309-12)
T14 M06 (MECHE5.3)
T08 M06 (MECHE3.7 MD)
T09 M06 (MECHE4.6 MD)
T19 M06 (MECHE MD D4.8)
T10 M06 (MECHE5.5 MD)
T05 M06 (MECHE MD D5.8)
T24 M06 (FRSURF50X5)
T02 M06 (VHM 309-12)
T03 M06 (VHM 309-6)
T06 M06 (VHM 309-3)
T23 M06 (CHANF8)
T22 M06 (CHANF3)


In [35]:
myProgOpTool=[]
toolNb=0
for i, line in enumerate(progLines):
    if i==0:
        myProgOpTool.append([line[0], line[1], line[2], line[3], myTools[0]])
        toolNb+=1
    else:
        secondStr=line[1].split(' ')[1:]
        myStr=' '.join(i for i in secondStr)
        if toolNb<len(myTools):
            if myStr == myTools[toolNb] :
                toolNb+=1    
        myProgOpTool.append([line[0], line[1], line[2], line[3], myTools[toolNb-1]])
del toolNb, i, line, myStr, secondStr, progLines      

In [36]:
DfProg=pd.DataFrame(myProgOpTool, columns=['Op_nb', 'Op_Name', 'BigOp_Nb', 'BigOp_Name', 'Tool_Name'])
DfProg.head()

,Op_nb,Op_Name,BigOp_Nb,BigOp_Name,Tool_Name
0,3380,N3380 X-152.678 Y20.567,3335,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)
1,3390,N3390 X-85.322,3335,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)
2,3400,N3400 X-62.072 Y43.857,3335,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)
3,3410,N3410 X-53.527 Y12.076,3335,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)
4,3420,N3420 G80 G00 Z5.,3335,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)


In [37]:
import pyarrow as pa 
import pyarrow.parquet as pq

myResTable=pa.Table.from_pandas(DfProg)
pq.write_table(myResTable, 'F:/Data_BachelorHES/3.Video_CNC/10.ProcessingResults/OriginalProgramLines.parquet')
del myResTable

In [22]:

#pip install rapidfuzz

## String matching with RapidFuzz

In [23]:
from rapidfuzz import fuzz, process
from tqdm import tqdm

#Extract the best candidates for a list of detected texts
def get_best_candidates(frame_txt, possible_matches):
    candidates = [process.extractOne(txt, possible_matches, scorer=fuzz.ratio) for txt in frame_txt if isinstance(txt, str)]
    return candidates

#iterate over each piece of OCR texts and compare results with the CNC program lines
results=[]
#choices is a partial list from myProgOpTool
subProgLines=myProgOpTool[0:30]
choices=[line[1] for line in subProgLines]
currentResult=(myFrameTexts[0][0], tuple(myFrameTexts[0][1:]), process.extractOne(myFrameTexts[0][2], choices, scorer=fuzz.ratio), 3380,1)
del subProgLines


#Choices contains the entire pool of strings to be matched
choices=[line[1] for line in myProgOpTool]

for myFrameText in tqdm(myFrameTexts, desc="Processing"):

    #previousResult contains the string matching from the previous frame and the program line number
    previousResult=(currentResult[2], currentResult[3])

    # myCandidates is a list of string matching results for each text detected in the frame
    # each element is a tuple with the string matching result and the score (and also the index of choices) 
    myCandidates=get_best_candidates(myFrameText, choices)

    #string_matches is a list of the string matching results
    string_matches=[candidate[0] for candidate in myCandidates]
    #If several strings matched are identical, we need only one
    uniqueMatch=set(string_matches)
    
    #bestCandidate is the best string matched with the highest score 
    bestCandidate=max(myCandidates, key=lambda x: x[1])
    
    #myNum is the program line number 
    myNum=int(bestCandidate[0].split(' ')[0].split('N')[-1])
    
    #currentResult all the relevant information : frame number, detected texts, best string matched, program line number, number of unique matches    
    currentResult=(myFrameText[0], tuple(myFrameText[1:]), bestCandidate, myNum, len(uniqueMatch))
    
    #If the current result score  is below 70, we keep the previous results
    if not bestCandidate[1]>70:
        currentResult=(myFrameText[0], tuple(myFrameText[1:]), previousResult[0],previousResult[1] ,len(uniqueMatch))
    #We keep the results in a global list results
    results.append(currentResult)
    
del bestCandidate, choices, currentResult, string_matches, myCandidates, myFrameText, myNum, previousResult, uniqueMatch


Processing: 100%|██████████| 55117/55117 [23:18<00:00, 39.40it/s] 


In [24]:

condensedRes=[[r[0], r[1], r[3], r[2][0]] for r in results]
opName=[line[1] for line in myProgOpTool]
tool=[line[4] for line in myProgOpTool]
bigOp=[line[3] for line in myProgOpTool]

for r in condensedRes:
    index=opName.index(r[3])
    r.append(bigOp[index])
    r.append(tool[index])
    
del bigOp, index, opName, tool, r   

In [25]:
DfRes=pd.DataFrame(condensedRes, columns=['Frame_Number', 'Line_Text_OCR','Line_Number', 'Line_Text_Program', 'Op_Name', 'Tool_Name'])

In [26]:
DfRes['Line_Text_OCR']=DfRes['Line_Text_OCR'].apply(lambda x: list(x))    

In [27]:
DfRes.head()

,Frame_Number,Line_Text_OCR,Line_Number,Line_Text_Program,Op_Name,Tool_Name
0,9965,"[N99003380, B380X-152.678Y20.567]",3380,N3380 X-152.678 Y20.567,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)
1,9970,[380X-152.678Y20.567],3380,N3380 X-152.678 Y20.567,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)
2,9975,[380X152.678720.567],3380,N3380 X-152.678 Y20.567,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)
3,9980,"[NO0003380, 380X-152.678Y20.567]",3380,N3380 X-152.678 Y20.567,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)
4,9985,"[NO0003380, 3380x-152.678120.567]",3380,N3380 X-152.678 Y20.567,(PRE-PERCAGE D4.2),T07 M06 (MECHE MD D4.2)


## Saving the result in .parquet file for future usage 

In [31]:
#pip install pyarrow

In [30]:
import pyarrow as pa 
import pyarrow.parquet as pq

myResTable=pa.Table.from_pandas(DfRes)
pq.write_table(myResTable, 'F:/Data_BachelorHES/3.Video_CNC/10.ProcessingResults/OCR_Program_Match_LabelledData.parquet')
del myResTable